In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [25]:
#Prepare data

trains_number = ({
    'KA MRI_BST':[801802,805806,809810,813814,817818,821822,825826,829830,833834,841842,845846,849850,853854,861862,
                  865866,869870,873874,877878,881882,885886,889890,893894,897898,901902,905906,909910,913914,917918,
                  921922,925926,929930,933934],
    'KA BST_MRI':[803804,807808,811812,815816,819820,823824,827828,831832,835836,843844,847848,851852,855856,863864,
                  867868,871872,875876,879880,883884,887888,890891,895896,899890,903904,907908,911912,915916,919920,
                  923924,927928,931932,935936]
})

relasi_list =({ 
    'Relasi MRI_BST':['MRI-SUDB','MRI-DU','MRI-BPR','MRI-RW','MRI-BST','SUDB-DU','SUDB-BPR','SUDB-RW','SUDB-BST','DU-BPR','DU-RW','DU-BST','RW-BPR','RW-BST','BPR-BST'],
    'Relasi BST_MRI':['BST-BPR','BST-RW','BST-DU','BST-SUDB','BST-MRI','BPR-RW','BPR-DU','BPR-SUDB','BPR-MRI','RW-DU','RW-SUDB','RW-MRI','DU-SUDB','DU-MRI','SUDB-MRI'],
})
tarif_list =({ 
    'Relasi':['MRI-SUDB','MRI-DU','MRI-BPR','MRI-RW','MRI-BST','SUDB-DU','SUDB-BPR','SUDB-RW','SUDB-BST','DU-BPR','DU-RW','DU-BST','RW-BPR','RW-BST','BPR-BST','BST-BPR','BST-RW','BST-DU','BST-SUDB','BST-MRI','BPR-RW','BPR-DU','BPR-SUDB','BPR-MRI','RW-DU','RW-SUDB','RW-MRI','DU-SUDB','DU-MRI','SUDB-MRI'],
    'Tarif Dasar' :[10000,10000,35000,35000,80000,10000,35000,35000,80000,25000,25000,70000,25000,35000,35000,35000,35000,70000,80000,80000,35000,25000,35000,35000,25000,35000,35000,10000,10000,10000],
})


departure_start_dates = datetime.strptime("20250201", "%Y%m%d")
departure_end_dates = datetime.strptime("20250228", "%Y%m%d")

departure_start_dates_mri=departure_start_dates
departure_start_dates_bst=departure_start_dates

data_mri_bst =[]
while departure_start_dates_mri <= departure_end_dates :
    payment_start_date = departure_start_dates_mri - timedelta(days=7)
    payment_date = pd.date_range(start=payment_start_date, end=departure_start_dates_mri)
    
    for no_ka_mri_bst in trains_number['KA MRI_BST']:
        for relasi_mri_bst in relasi_list['Relasi MRI_BST']:
            for payment in payment_date:
                data_mri_bst.append({
                    'No KA' : no_ka_mri_bst,
                    'Relasi' : relasi_mri_bst,
                    'Payment Date': payment.date(),
                    'Trip Date' : departure_start_dates_mri.date(),
                })
    departure_start_dates_mri += timedelta(days=1)
    
data_bst_mri =[]
while departure_start_dates_bst <= departure_end_dates :
    payment_start_date = departure_start_dates_bst - timedelta(days=7)
    payment_date = pd.date_range(start=payment_start_date, end=departure_start_dates_bst)
    
    for no_ka_bst_mri in trains_number['KA BST_MRI']:
        for relasi_bst_mri in relasi_list['Relasi BST_MRI']:
            for payment in payment_date:
                data_bst_mri.append({
                    'No KA' : no_ka_bst_mri,
                    'Relasi' : relasi_bst_mri,
                    'Payment Date': payment.date(),
                    'Trip Date' : departure_start_dates_bst.date(),
                })
    
    departure_start_dates_bst += timedelta(days=1)

df_bst_mri=pd.DataFrame(data_bst_mri)
df_mri_bst=pd.DataFrame(data_mri_bst)

df=pd.concat([df_mri_bst,df_bst_mri],ignore_index=True)

df_tarif_list=pd.DataFrame(tarif_list)

df_tarif =pd.merge(df,df_tarif_list, how='inner', on ='Relasi')

df_tarif['Payment Date']=pd.to_datetime(df_tarif['Payment Date'])
df_tarif['Trip Date']=pd.to_datetime(df_tarif['Trip Date'])
df_tarif['Selisih Hari Pembelian']=(df_tarif['Trip Date'] - df_tarif['Payment Date']).dt.days

# === VERSI VECTORIZE UNTUK TARIF SALES ===

# Relasi yang dapat diskon
diskon_relasi = ['MRI-BST', 'SUDB-BST', 'DU-BST', 'BST-MRI', 'BST-SUDB', 'BST-DU']

# Boolean mask untuk baris yg kena diskon
mask_diskon = df_tarif['Relasi'].isin(diskon_relasi)

# Diskon 5% * Selisih Hari
diskon_persen = 0.05 * df_tarif['Selisih Hari Pembelian']

# Hitung tarif setelah diskon
tarif_sales = df_tarif['Tarif Dasar'] * (1 - diskon_persen)

# Tarif non-diskon tetap sama
tarif_sales[~mask_diskon] = df_tarif.loc[~mask_diskon, 'Tarif Dasar']

# Minimum tarif setelah diskon Rp50.000 untuk relasi diskon
tarif_sales[mask_diskon] = np.maximum(tarif_sales[mask_diskon], 50000)

# Bulatkan ke ribuan terdekat
df_tarif['Tarif Sales'] = (np.round(tarif_sales / 1000) * 1000).astype(int)

df_tarif


,No KA,Relasi,Payment Date,Trip Date,Tarif Dasar,Selisih Hari Pembelian,Tarif Sales
0,801802,MRI-SUDB,2025-01-25,2025-02-01,10000,7,10000
1,801802,MRI-SUDB,2025-01-26,2025-02-01,10000,6,10000
2,801802,MRI-SUDB,2025-01-27,2025-02-01,10000,5,10000
3,801802,MRI-SUDB,2025-01-28,2025-02-01,10000,4,10000
4,801802,MRI-SUDB,2025-01-29,2025-02-01,10000,3,10000
...,...,...,...,...,...,...,...
215035,935936,SUDB-MRI,2025-02-24,2025-02-28,10000,4,10000
215036,935936,SUDB-MRI,2025-02-25,2025-02-28,10000,3,10000
215037,935936,SUDB-MRI,2025-02-26,2025-02-28,10000,2,10000
215038,935936,SUDB-MRI,2025-02-27,2025-02-28,10000,1,10000
